# Halo 4 facial animations.

Using principal component analysis to compress vertex animation.

Notebook by Jerome Eippers, 2024

In [ ]:
%matplotlib widget
import ipyanimlab as lab
import numpy as np
from sklearn import decomposition
from ipywidgets import widgets, interact
from matplotlib import pyplot as plt

viewer = lab.Viewer(move_speed=5, width=1280, height=720)
viewer.set_time_of_day(38)

In [ ]:
material = lab.Material('white',albedo=np.array([1,1,1], dtype=np.float32), roughness=0.5, metallic=0.0, reflectance=.1, emissive=0.0)

## Load data
Let's load animation data as pure stream of vertex positions.  

In [ ]:
import pickle
with open('animated_face.dat', 'rb') as f:
    indices, normals, frames = pickle.load(f)
    
indices = np.array(indices, dtype=np.uint16)
normals = np.array(normals, dtype=np.float32)
frames = np.array(frames, dtype=np.float32)

## ---------------------- DATA -------------------------
## indices = [triangle count, 3] the list of vertex indices for each triangle
## normals = [number of vertex, 3] the list of xyz component for each vertex normals
## frames = [frame count, number of vertex, 3] the xyz position for each vertex for each frame of animation.

display(('indices', indices.shape))
display(('normals', normals.shape))
display(('frames', frames.shape))

## Build streaming shader
We need to create a dynamic vertex buffer to stream the facial animation, and a new shader to read it.

In [ ]:
vbo = viewer.create_buffer_ext(src_data=frames[0, ...].astype(np.float32).flatten(), usage='DYNAMIC_DRAW')
vbo_normals = viewer.create_buffer_ext(src_data=normals.astype(np.float32).flatten())

In [ ]:
vao = viewer.create_vertex_array_ext(
            None,
            [(vbo, '3f32', 0),
            (vbo_normals, '3f32', 1)],
            indices
        )

In [ ]:
vs = """#version 300 es
//the ViewBlock that is automatically filled by ipywebgl
layout(std140) uniform ViewBlock
{
    mat4 u_cameraMatrix;          //the camera matrix in world space
    mat4 u_viewMatrix;            //the inverse of the camera matrix
    mat4 u_projectionMatrix;      //the projection matrix
    mat4 u_viewProjectionMatrix;  //the projection * view matrix
};

in vec3 in_vert;
in vec3 in_normals;

out vec4 v_viewposition;
out vec4 v_viewnormal;
out vec4 v_vertcolor;

void main() {
    vec4 pos = vec4(in_vert, 1.0);
    vec4 normal = vec4(in_normals, 0.0);
    
    v_viewposition = pos * u_viewMatrix;
    v_viewnormal = normal * u_viewMatrix;
    gl_Position = v_viewposition * u_projectionMatrix;

    v_vertcolor = vec4(1,1,1,1);
  }
"""

fs = """#version 300 es
precision highp float;

uniform vec3 u_color;
uniform vec4 u_material;
in vec4 v_viewposition;
in vec4 v_viewnormal;
in vec4 v_vertcolor;


layout(location=0) out vec4 color;
layout(location=1) out vec4 material;
layout(location=2) out vec4 viewPosition;
layout(location=3) out vec4 viewNormal;

void main() {
    color = vec4(u_color * v_vertcolor.rgb, v_vertcolor.a);
    material = u_material;
    viewPosition = vec4(v_viewposition.xyz, 1.0);
    viewNormal = vec4(normalize(v_viewnormal.xyz), 0.0);
}
"""
prog = viewer.create_program_ext(
            vs,
            fs,
            {
                'in_vert' : 0,
                'in_normals' : 1,
            }
        )

## Render vertex stream

In [ ]:
def render(frame):
    viewer.bind_buffer(buffer=vbo)
    viewer.buffer_data(src_data=frames[frame, ...].astype(np.float32).flatten(), usage='DYNAMIC_DRAW')
    
    viewer.begin_shadow()
    viewer.end_shadow()

    viewer.begin_display()
    viewer.draw_ground()

    viewer.use_program(prog)
    viewer.bind_vertex_array(vao)
    viewer.uniform('u_color', material.albedo)
    viewer.uniform('u_material', material.material_uniform)
    viewer.draw_elements('TRIANGLES', indices.shape[0]*3, 'UNSIGNED_SHORT', 0)

    viewer.end_display()
    viewer.execute_commands()
    
interact(
    render, 
    frame=lab.Timeline(min=0, max=frames.shape[0]-1)
)
viewer

## The issue

The size of the data to stream is huge. We will need to find a way to compress this data.

In [ ]:
display(f'animation memory size : {frames.size * 4 / 1024 / 1024} Mb')

## PCA

Let's use a PCA to reduce the number of dimensions. In this case we will reduce the 220 frames of animations into just 7 'frames'

In [ ]:
data = frames.reshape(220, -1)

pca = decomposition.PCA(n_components=7)
pca.fit(data)

The PCA will give us 7 'frames' with a full head

In [ ]:
pca.components_.shape

### Display the components

Let's see the different components that where extracted from the 220 frames of animation.  
The PCA did extract the mean before computing the components, so we have to add it to each components.

In [ ]:
def render(pose, mult=50.0):
    
    v = pca.components_[pose, :]*mult + pca.mean_
    
    viewer.begin_shadow()
    viewer.end_shadow()

    viewer.begin_display()
    viewer.draw_ground()

    viewer.use_program(prog)
    
    viewer.bind_buffer(buffer=vbo)
    viewer.buffer_data(src_data=v.astype(np.float32).flatten(), usage='DYNAMIC_DRAW')
    
    viewer.bind_vertex_array(vao)
    viewer.uniform('u_color', material.albedo)
    viewer.uniform('u_material', material.material_uniform)
    viewer.draw_elements('TRIANGLES', indices.shape[0]*3, 'UNSIGNED_SHORT', 0)

    viewer.end_display()
    viewer.execute_commands()
    
interact(
    render, 
    pose=widgets.IntSlider(min=0, max= pca.components_.shape[0]-1)
)
viewer

### Compute the animation

Using the PCA, we can now project each frames into the 7 components.  
So each frame of animations in now composed of 7 'weights' instead of 60k vertices.

In [ ]:
anim_pca = pca.transform(data)

### Render using PCA

We wil use the inverse_transform to convert back each pca frames into a full head.  
And we render both head next to each other to compare.

In [ ]:
def render(frame):
    
    display(anim_pca[frame])
    anim = anim_pca[frame]
    
    v = pca.inverse_transform(anim).reshape(-1, 3)
    v[:, 0] += 30
    
    viewer.begin_shadow()
    viewer.end_shadow()

    viewer.begin_display()
    viewer.draw_ground()

    viewer.use_program(prog)
    
    viewer.bind_buffer(buffer=vbo)
    viewer.buffer_data(src_data=frames[frame, ...].astype(np.float32).flatten(), usage='DYNAMIC_DRAW')
    
    viewer.bind_vertex_array(vao)
    viewer.uniform('u_color', material.albedo)
    viewer.uniform('u_material', material.material_uniform)
    viewer.draw_elements('TRIANGLES', indices.shape[0]*3, 'UNSIGNED_SHORT', 0)
    
    viewer.bind_buffer(buffer=vbo)
    viewer.buffer_data(src_data=v.astype(np.float32).flatten(), usage='DYNAMIC_DRAW')
    
    viewer.bind_vertex_array(vao)
    viewer.uniform('u_color', material.albedo)
    viewer.uniform('u_material', material.material_uniform)
    viewer.draw_elements('TRIANGLES', indices.shape[0]*3, 'UNSIGNED_SHORT', 0)

    viewer.end_display()
    viewer.execute_commands()
    
interact(
    render, 
    frame=lab.Timeline(min=0, max=frames.shape[0]-1)
)
viewer

## Final Render

Now we can write a shader that will take the 8 heads (the mean plus the 7 components) and that will use the 7 weights to compute the final mesh.

In [ ]:
vs_pca = """#version 300 es
precision highp float;
//the ViewBlock that is automatically filled by ipywebgl
layout(std140) uniform ViewBlock
{
    mat4 u_cameraMatrix;          //the camera matrix in world space
    mat4 u_viewMatrix;            //the inverse of the camera matrix
    mat4 u_projectionMatrix;      //the projection matrix
    mat4 u_viewProjectionMatrix;  //the projection * view matrix
};

in vec3 in_vert;
in vec3 in_vert_0;
in vec3 in_vert_1;
in vec3 in_vert_2;
in vec3 in_vert_3;
in vec3 in_vert_4;
in vec3 in_vert_5;
in vec3 in_vert_6;
in vec3 in_normals;

out vec4 v_viewposition;
out vec4 v_viewnormal;
out vec4 v_vertcolor;

uniform float u_pca_0;
uniform float u_pca_1;
uniform float u_pca_2;
uniform float u_pca_3;
uniform float u_pca_4;
uniform float u_pca_5;
uniform float u_pca_6;

void main() {
    vec3 accumulated = in_vert;
    accumulated += vec3(u_pca_0) * in_vert_0;
    accumulated += vec3(u_pca_1) * in_vert_1;
    accumulated += vec3(u_pca_2) * in_vert_2;
    accumulated += vec3(u_pca_3) * in_vert_3;
    accumulated += vec3(u_pca_4) * in_vert_4;
    accumulated += vec3(u_pca_5) * in_vert_5;
    accumulated += vec3(u_pca_6) * in_vert_6;
    
    vec4 pos = vec4(accumulated, 1.0);
    vec4 normal = vec4(in_normals, 0.0);
    
    v_viewposition = pos * u_viewMatrix;
    v_viewnormal = normal * u_viewMatrix;
    gl_Position = v_viewposition * u_projectionMatrix;

    v_vertcolor = vec4(1,1,1,1);
  }
"""

# build the buffers
vbo_mean = viewer.create_buffer_ext(src_data=pca.mean_.astype(np.float32).flatten())
vbo_pca_0 = viewer.create_buffer_ext(src_data=pca.components_[0, :].astype(np.float32).flatten())
vbo_pca_1 = viewer.create_buffer_ext(src_data=pca.components_[1, :].astype(np.float32).flatten())
vbo_pca_2 = viewer.create_buffer_ext(src_data=pca.components_[2, :].astype(np.float32).flatten())
vbo_pca_3 = viewer.create_buffer_ext(src_data=pca.components_[3, :].astype(np.float32).flatten())
vbo_pca_4 = viewer.create_buffer_ext(src_data=pca.components_[4, :].astype(np.float32).flatten())
vbo_pca_5 = viewer.create_buffer_ext(src_data=pca.components_[5, :].astype(np.float32).flatten())
vbo_pca_6 = viewer.create_buffer_ext(src_data=pca.components_[6, :].astype(np.float32).flatten())

prog_pca = viewer.create_program_ext(
            vs_pca,
            fs,
            {
                'in_vert' : 0,
                'in_normals' : 1,
                'in_vert_0' : 2,
                'in_vert_1' : 3,
                'in_vert_2' : 4,
                'in_vert_3' : 5,
                'in_vert_4' : 6,
                'in_vert_5' : 7,
                'in_vert_6' : 8,
            }
        )

vao_pca = viewer.create_vertex_array_ext(
            prog_pca,
            [(vbo_mean, '3f32', 0),
            (vbo_normals, '3f32', 1),
            (vbo_pca_0, '3f32', 2),
            (vbo_pca_1, '3f32', 3),
            (vbo_pca_2, '3f32', 4),
            (vbo_pca_3, '3f32', 5),
            (vbo_pca_4, '3f32', 6),
            (vbo_pca_5, '3f32', 7),
            (vbo_pca_6, '3f32', 8)],
            indices
        )

In [ ]:
def render(frame):
    anim = anim_pca[frame]
    
    viewer.begin_shadow()
    viewer.end_shadow()

    viewer.begin_display()
    viewer.draw_ground()

    viewer.use_program(prog_pca)
    
    viewer.bind_vertex_array(vao_pca)
    viewer.uniform('u_color', material.albedo)
    viewer.uniform('u_material', material.material_uniform)
    viewer.uniform('u_pca_0', anim[0:1])
    viewer.uniform('u_pca_1', anim[1:2])
    viewer.uniform('u_pca_2', anim[2:3])
    viewer.uniform('u_pca_3', anim[3:4])
    viewer.uniform('u_pca_4', anim[4:5])
    viewer.uniform('u_pca_5', anim[5:6])
    viewer.uniform('u_pca_6', anim[6:])
    viewer.draw_elements('TRIANGLES', indices.shape[0]*3, 'UNSIGNED_SHORT', 0)
    viewer.end_display()
    viewer.execute_commands()
    
interact(
    render, 
    frame=lab.Timeline(min=0, max=anim_pca.shape[0]-1)
)
viewer